In [1]:
data_var = '2024-01-02'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8650,2024-01-02,Eua Ncaa,23:00,SMU Mustangs,Charlotte,1.30,3.47,134.5,1.83,1.83,-8.5,1.91,1.80,A7BqlfPK,0.769231,0.288184,0.546448,0.546448,0.057415,132.586,27.418435,0.206797,1.8,1.643168,0.912871,131.56,1.770,0.525119,0.296677,77.0,201.356,176.026439,0.874205,0.6,1.341641,2.236068,119.25,3.226,3.379842,1.047688,-17.0,92,75,1.43,1.59,109.926,211.488,0.643363,0.000000,0.041931,-1.88,-0.376,-0.797872,0.000000,0.0,0.000000,-2.90,-0.580,-4.258621,0.000000,0.0,0.000000
8651,2024-01-02,Europa Liga Dos Campeões,14:30,PAOK,Tofas,1.68,2.16,162.5,1.87,1.91,-5.5,2.04,1.69,fecVwq2p,0.595238,0.462963,0.534759,0.523560,0.058201,180.628,57.040770,0.315791,1.2,1.643168,1.369306,141.41,2.512,0.942215,0.375086,-34.0,235.990,121.661794,0.515538,0.6,1.341641,2.236068,202.50,2.774,1.366411,0.492578,-21.0,79,75,1.79,2.70,0.000,0.000,0.176777,0.014965,0.132701,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8652,2024-01-02,Eua Ncaa,23:00,Illinois,Northwestern,1.45,2.74,143.5,1.83,1.83,-6.5,1.91,1.80,veDHib3Q,0.689655,0.364964,0.546448,0.546448,0.054619,152.648,66.430789,0.435189,1.8,1.643168,0.912871,100.01,1.876,0.710619,0.378795,13.0,81.474,12.632806,0.155053,2.4,1.341641,0.559017,75.48,1.152,0.206083,0.178891,70.0,73,74,1.37,1.02,112.188,0.000,0.435402,0.000000,0.041931,0.26,0.052,8.653846,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8653,2024-01-02,Eua Ncaa,23:00,Duke,Syracuse,1.06,8.75,151.5,1.83,1.83,-14.5,1.73,0.00,dtPIbPbd,0.943396,0.114286,0.546448,0.546448,0.057682,102.624,27.666760,0.269593,1.8,1.643168,0.912871,145.86,1.320,0.355528,0.269339,44.0,144.620,47.165661,0.326135,3.0,0.000000,0.000000,162.00,1.772,0.568085,0.320589,63.0,78,81,1.87,2.00,120.038,0.000,1.108594,0.000000,1.414214,2.26,0.452,0.132743,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8654,2024-01-02,Eua Ncaa,23:00,UTSA Roadrunners,UAB,2.74,1.45,158.5,1.80,1.91,5.5,1.83,1.83,lbAmmEvR,0.364964,0.689655,0.555556,0.523560,0.054619,195.034,95.354169,0.488910,1.2,1.643168,1.369306,168.92,2.554,1.407633,0.551148,-3.0,125.688,34.100463,0.271310,1.8,1.643168,0.912871,126.90,1.548,0.478247,0.308945,-5.0,103,90,1.64,1.41,129.280,0.000,0.435402,0.041931,0.000000,-0.18,-0.036,-48.333333,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8716,2024-01-02,China Cba,08:35,Shenzhen,Shanghai,1.57,2.34,211.5,1.83,1.91,-4.5,1.95,1.75,SG3gwz1Q,0.636943,0.427350,0.546448,0.523560,0.064293,191.974,84.046132,0.437800,2.4,1.341641,0.559017,123.60,1.830,0.955170,0.521951,2.0,271.432,105.708721,0.389448,1.8,1.643168,0.912871,193.98,2.410,1.020809,0.423572,14.0,120,106,1.03,1.83,162.032,281.496,0.278502,0.030251,0.076444,2.41,0.482,1.182573,0.768176,0.8,0.031824,0.85,0.170,7.882353,0.541160,0.5,-0.041160
8717,2024-01-02,China Cba,08:35,Shandong,Ningbo Rockets,1.20,4.35,200.5,1.85,1.87,-11.5,1.94,1.76,UPkLYg9s,0.833333,0.229885,0.540541,0.534759,0.063218,291.294,122.879375,0.421840,0.6,1.341641,2.236068,192.00,2.938,1.287777,0.438318,-27.0,344.380,139.021125,0.403685,1.2,1.643168,1.369

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
8651,14:30,Europa Liga Dos Campeões,PAOK,Tofas,1.68,Back Home
8660,22:00,Eua Ncaa,Murray State,Illinois (Chi.),1.69,Back Home
8671,09:30,Turquia Copa Da Turquia,Emlak Konut F,Ormanspor F,2.37,Back Home
8675,14:00,Israel Liga Leumit,Ramat Hasharon,Maccabi Raanana,2.32,Back Home
8676,13:35,Israel Liga Leumit,Maccabi Rishon,Elitzor Netania,1.74,Back Home
8677,13:30,Israel Liga Leumit,Hapoel Bnei Kfar Kasem,Ironi Nahariya,2.15,Back Home
8681,13:30,Finlândia Korisliiga,Salon Vilpas,BC Nokia,1.80,Back Home
8683,21:00,Eua Ncaa,Pittsburgh,North Carolina,2.18,Back Home
8688,15:00,Alemanha Pro A,Giessen,Trier,1.64,Back Home
8691,16:05,Europa Euroliga,Maccabi Tel Aviv,Monaco,2.28,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
8671,09:30,Turquia Copa Da Turquia,Emlak Konut F,Ormanspor F,1.85,Back Home
8675,14:00,Israel Liga Leumit,Ramat Hasharon,Maccabi Raanana,1.85,Back Home
8683,21:00,Eua Ncaa,Pittsburgh,North Carolina,1.83,Back Home


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
8651,14:30,Europa Liga Dos Campeões,PAOK,Tofas,1.68,Back Home
8692,15:30,Alemanha Pro A,G. Dusseldorf,Bremerhaven,1.90,Back Home
8716,08:35,China Cba,Shenzhen,Shanghai,1.57,Back Home
8718,13:00,Catar Liga De Basquete Do Catar,Al Sadd,Al Gharafa,2.26,Back Home
8719,21:00,Eua Ncaa,Ohio,Toledo,1.69,Back Home
